# 1.1 Загрузка библиотек и парсинг данных

In [ ]:
# Импортируем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import nltk
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
# Отключаю предупреждения
warnings.filterwarnings('ignore')

In [ ]:
# !gdown 1jOrdR6Kl42cEtsmqlo8e-gpRqu9dwmGO
# !gdown 107vN40gSh3r10P3iAhSEx9Bb8nvJZNHc
# !gdown 1FI4jwywSQ0ySuY9YASAhrUg9JLbNDW76
# !gdown 1vVf2jJSp39spTUuMaOBi3RG2nTbRZhv3

In [ ]:
# # Загрузка данных
# test_stances = pd.read_csv("test_stances_unlebeledb.csv", index_col='Unnamed: 0')
# test_bodies = pd.read_csv("test_bodies.csv", index_col='Unnamed: 0')
# train_bodies = pd.read_csv("train_bodies.csv", index_col='Unnamed: 0')
# train_stances = pd.read_csv("train_stances.csv", index_col='Unnamed: 0')

In [ ]:
# # Объединения двух csv-файлов тренирвочной выборки в один файл
# count=0
# for i in tqdm(range(train_stances.shape[0])):
#     for j in range(train_bodies.shape[0]):
#         if train_bodies.loc[j,'Body ID']==train_stances.loc[i,'Body ID']:
#             train_stances.loc[i,'articleBody'] = train_bodies.loc[j,'articleBody']
#             train_stances.loc[i,'articleBody1'] = train_bodies.loc[j,'articleBody1']
#             train_stances.loc[i,'articleBody2'] = train_bodies.loc[j,'articleBody2']
#     train_stances.to_csv('data_combined.csv',index=False)

In [ ]:
# # Объединения двух csv-файлов тестовой выборки в один файл
# for i in tqdm(range(test_stances.shape[0])):
#     for j in range(test_bodies.shape[0]):
#         if train_bodies.loc[j,'Body ID'] == train_stances.loc[i,'Body ID']:
#             test_stances.loc[i,'articleBody'] = test_bodies.loc[j,'articleBody']
#             test_stances.loc[i,'articleBody1'] = test_bodies.loc[j,'articleBody1']
#             test_stances.loc[i,'articleBody2'] = test_bodies.loc[j,'articleBody2']
#     test_stances.to_csv('data_test_combined.csv',index=False)

In [ ]:
!gdown 1011cPGtaU1awVbf9QKVDzIZ9tKBc-qqo

Downloading...
From: https://drive.google.com/uc?id=1011cPGtaU1awVbf9QKVDzIZ9tKBc-qqo
To: /content/data_combined.csv
100% 31.5M/31.5M [00:00<00:00, 151MB/s]


In [ ]:
data = pd.read_csv('data_combined.csv') # Запишем новые данные в файл
data.head() # Посмотрим на данные

,Body ID,Headline,Stance,Headline1,Headline2,articleBody,articleBody1,articleBody2
0,1,"['российский', 'бюджет', 'марте', 'недополучил...",agree,российский бюджет в марте недополучил более 30...,российский бюджет март недополучать миллиард р...,Разница между ожидаемыми по итогам марта нефте...,разница ожидаемыми итогам марта нефтегазовыми ...,разница ожидать итог март нефтегазовый доход ф...
1,2,"['банк', 'россии', 'решил', 'снизить', 'ключев...",agree,банк россии решил снизить ключевую ставку с 20...,банк россия решать снижать ключевой ставка,Совет директоров Банка России решил снизить кл...,совет директоров банка россии решил снизить кл...,совет директор банк россия решать снижать ключ...
2,3,"['мыс', 'идокопас', 'нато', 'назвали', 'первую...",disagree,мыс идокопас в нато назвали первую цель для н...,мыс идокопас нато называть первый цель начало ...,В НАТО составили наступательный план действий ...,нато составили наступательный план действий за...,нато составлять наступательный план действие з...
3,4,"['украине', 'прошли', 'празднования', '300', '...",disagree,на украине прошли празднования 300 летия побед...,украина проходить празднование летие победа ка...,Президент Владимир Зеленский принял в Полтаве ...,президент владимир зеленский принял полтаве па...,президент владимир зеленский принимать полтава...
4,5,"['минобороны', 'рф', 'заявило', 'ударах', 'рак...",agree,минобороны рф заявило об ударах ракетами «кали...,минобороны заявлять удар ракета калибр запорож...,"Минобороны России объявило, что ракетами больш...",минобороны россии объявило ракетами большой да...,минобороны россия объявлять ракета большой дал...


In [ ]:
!gdown 1SXQim_SWf39_3ZfoY4aCSj4biZqVceor

Downloading...
From: https://drive.google.com/uc?id=1SXQim_SWf39_3ZfoY4aCSj4biZqVceor
To: /content/data_test_combined.csv
100% 7.86M/7.86M [00:00<00:00, 68.7MB/s]


In [ ]:
data_test = pd.read_csv('data_test_combined.csv') # Запишем новые данные в файд
data_test.head() # Посмотрим на данные

,Body ID,label,Headline,Headline1,Headline2,articleBody,articleBody1,articleBody2
0,1,0,"['лукашенко', 'пригрозил', 'литовским', 'танка...",лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозить литовский танк белорусски...,"Президент Беларуси Александр Лукашенко заявил,...",президент беларуси александр лукашенко заявил ...,президент беларусь александр лукашенко заявлят...
1,2,1,"['российские', 'компании', 'оказались', 'опасн...",российские компании оказались в опасности из з...,российский компания оказываться опасность глоб...,Несколько крупнейших российских компаний оказа...,несколько крупнейших российских компаний оказа...,несколько крупный российский компания оказыват...
2,3,0,"['лукашенко', 'объявил', 'отмене', 'выборов', ...",лукашенко объявил об отмене выборов из за напа...,лукашенко объявлять отмена выборы нападение ин...,Президент Белоруссии Александр Лукашенко объяв...,президент белоруссии александр лукашенко объяв...,президент белоруссия александр лукашенко объяв...
3,4,1,"['«роснефть»', 'стала', 'лидером', 'объему', '...",«роснефть» стала лидером по объему биржевых пр...,роснефть становиться лидер объем биржевой прод...,С начала июля нефтяные компании заметно увелич...,начала июля нефтяные компании заметно увеличил...,начинать июль нефтяной компания заметно увелич...
4,5,1,"['псковской', 'области', 'дадут', 'десятки', '...",псковской области дадут десятки миллионов рубл...,псковский область давать десяток миллион рубль...,Псковская область получит 189 миллионов рублей...,псковская область получит 189 миллионов рублей...,псковский область получать миллион рубль ремон...


Данные тренировочной/тестовой выборки спарсены в один файл и готовы к последующей работе с ними.

# 1.2 Анализ данных

In [ ]:
data.info() # Выведем информацию

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4408 entries, 0 to 4407
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Body ID       4408 non-null   int64 
 1   Headline      4408 non-null   object
 2   Stance        4408 non-null   object
 3   Headline1     4408 non-null   object
 4   Headline2     4408 non-null   object
 5   articleBody   4408 non-null   object
 6   articleBody1  4407 non-null   object
 7   articleBody2  4407 non-null   object
dtypes: int64(1), object(7)
memory usage: 275.6+ KB


In [ ]:
data_test.info() # Выведем информацию

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1101 entries, 0 to 1100
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Body ID       1101 non-null   int64 
 1   label         1101 non-null   int64 
 2   Headline      1101 non-null   object
 3   Headline1     1101 non-null   object
 4   Headline2     1101 non-null   object
 5   articleBody   1101 non-null   object
 6   articleBody1  1101 non-null   object
 7   articleBody2  1101 non-null   object
dtypes: int64(2), object(6)
memory usage: 68.9+ KB


Информация о данных в тренировочной/тестовой выборки:
<li>4408/1101 записей</li>
<li>7/7 колонок</li>
<li>1/2 признака типа 'int64'</li>
<li>7/6 признака тиа 'object'</li>
<li>0/0 пропусков</li>

Обозначение признаков:

<b>Unnamed: 0</b> - Номер строки

<b>Body ID</b> - идентификатор id новости

<b>Stance</b> - правдивость новости

<b>Headline</b> - оригинальное название новостей

<b>Headline1</b> - название новостей после чистки

<b>Headline2</b> - название новостей после обработки модулем PyMystem3

<b>articleBody</b> - оригинальное содержание новостей

<b>articleBody1</b> -содержание новостей после чистки

<b>articleBody2</b> - содержание новостей после обработки модулем PyMystem3

# 1.3 Обработка данных и подкачка словаря слов

In [ ]:
# Кодируем котегориальную переменную
data['Stance'] = data['Stance'].replace(['agree', 'disagree'], [1, 0]).astype('int')

In [ ]:
data # Посмотрим на датафрейм

,Body ID,Headline,Stance,Headline1,Headline2,articleBody,articleBody1,articleBody2
0,1,"['российский', 'бюджет', 'марте', 'недополучил...",1,российский бюджет в марте недополучил более 30...,российский бюджет март недополучать миллиард р...,Разница между ожидаемыми по итогам марта нефте...,разница ожидаемыми итогам марта нефтегазовыми ...,разница ожидать итог март нефтегазовый доход ф...
1,2,"['банк', 'россии', 'решил', 'снизить', 'ключев...",1,банк россии решил снизить ключевую ставку с 20...,банк россия решать снижать ключевой ставка,Совет директоров Банка России решил снизить кл...,совет директоров банка россии решил снизить кл...,совет директор банк россия решать снижать ключ...
2,3,"['мыс', 'идокопас', 'нато', 'назвали', 'первую...",0,мыс идокопас в нато назвали первую цель для н...,мыс идокопас нато называть первый цель начало ...,В НАТО составили наступательный план действий ...,нато составили наступательный план действий за...,нато составлять наступательный план действие з...
3,4,"['украине', 'прошли', 'празднования', '300', '...",0,на украине прошли празднования 300 летия побед...,украина проходить празднование летие победа ка...,Президент Владимир Зеленский принял в Полтаве ...,президент владимир зеленский принял полтаве па...,президент владимир зеленский принимать полтава...
4,5,"['минобороны', 'рф', 'заявило', 'ударах', 'рак...",1,минобороны рф заявило об ударах ракетами «кали...,минобороны заявлять удар ракета калибр запорож...,"Минобороны России объявило, что ракетами больш...",минобороны россии объявило ракетами большой да...,минобороны россия объявлять ракета большой дал...
...,...,...,...,...,...,...,...,...
4403,4404,"['чиновница', 'призвавшая', '«найти', 'покарат...",0,чиновница призвавшая «найти и покарать предате...,чиновница призывать находить покарать предател...,Заместитель министра просвещения Ирина Карепин...,заместитель министра просвещения ирина карепин...,заместитель министр просвещение ирина карепин ...
4404,4405,"['«включим', 'программу', 'киселёва', 'покажем...",0,«включим программу киселёва и покажем телевизо...,включать программа киселев показывать телевизо...,Глава штабов Навального Леонид Волков призвал ...,глава штабов навального леонид волков призвал ...,глава штаб навальный леонид волк призывать сво...
4405,4406,"['китайские', 'власти', 'могут', 'казнить', 'з...",0,китайские власти могут казнить задержанного по...,китайский власть мочь казнить задержанный подо...,Один из менеджеров стройки космодрома Восточны...,менеджеров стройки космодрома восточный взят с...,менеджер стройка космодром восточный взять стр...
4406,4407,"['луганской', 'области', 'переполнены', 'морги...",1,в луганской области переполнены морги хранить...,луганский область переполнять морг хранить тел...,Морги Луганской области переполнены телами пог...,морги луганской области переполнены телами пог...,морг луганский область переполнять тело погибш...


In [ ]:
# Сделаем столбец "Body ID" индексом
data = data.set_index("Body ID")
data

,Headline,Stance,Headline1,Headline2,articleBody,articleBody1,articleBody2
Body ID,,,,,,,
1,"['российский', 'бюджет', 'марте', 'недополучил...",1,российский бюджет в марте недополучил более 30...,российский бюджет март недополучать миллиард р...,Разница между ожидаемыми по итогам марта нефте...,разница ожидаемыми итогам марта нефтегазовыми ...,разница ожидать итог март нефтегазовый доход ф...
2,"['банк', 'россии', 'решил', 'снизить', 'ключев...",1,банк россии решил снизить ключевую ставку с 20...,банк россия решать снижать ключевой ставка,Совет директоров Банка России решил снизить кл...,совет директоров банка россии решил снизить кл...,совет директор банк россия решать снижать ключ...
3,"['мыс', 'идокопас', 'нато', 'назвали', 'первую...",0,мыс идокопас в нато назвали первую цель для н...,мыс идокопас нато называть первый цель начало ...,В НАТО составили наступательный план действий ...,нато составили наступательный план действий за...,нато составлять наступательный план действие з...
4,"['украине', 'прошли', 'празднования', '300', '...",0,на украине прошли празднования 300 летия побед...,украина проходить празднование летие победа ка...,Президент Владимир Зеленский принял в Полтаве ...,президент владимир зеленский принял полтаве па...,президент владимир зеленский принимать полтава...
5,"['минобороны', 'рф', 'заявило', 'ударах', 'рак...",1,минобороны рф заявило об ударах ракетами «кали...,минобороны заявлять удар ракета калибр запорож...,"Минобороны России объявило, что ракетами больш...",минобороны россии объявило ракетами большой да...,минобороны россия объявлять ракета большой дал...
...,...,...,...,...,...,...,...
4404,"['чиновница', 'призвавшая', '«найти', 'покарат...",0,чиновница призвавшая «найти и покарать предате...,чиновница призывать находить покарать предател...,Заместитель министра просвещения Ирина Карепин...,заместитель министра просвещения ирина карепин...,заместитель министр просвещение ирина карепин ...
4405,"['«включим', 'программу', 'киселёва', 'покажем...",0,«включим программу киселёва и покажем телевизо...,включать программа киселев показывать телевизо...,Глава штабов Навального Леонид Волков призвал ...,глава штабов навального леонид волков призвал ...,глава штаб навальный леонид волк призывать сво...
4406,"['китайские', 'власти', 'могут', 'казнить', 'з...",0,китайские власти могут казнить задержанного по...,китайский власть мочь казнить задержанный подо...,Один из менеджеров стройки космодрома Восточны...,менеджеров стройки космодрома восточный взят с...,менеджер стройка космодром восточный взять стр...


In [ ]:
# Распарсим столбец "Headline"
data['Headline'] = data['Headline'].apply(lambda x: "".join(x.replace("[", "").replace("]", "").replace("'", "").split(',')))
data

,Headline,Stance,Headline1,Headline2,articleBody,articleBody1,articleBody2
Body ID,,,,,,,
1,российский бюджет марте недополучил 300 миллиа...,1,российский бюджет в марте недополучил более 30...,российский бюджет март недополучать миллиард р...,Разница между ожидаемыми по итогам марта нефте...,разница ожидаемыми итогам марта нефтегазовыми ...,разница ожидать итог март нефтегазовый доход ф...
2,банк россии решил снизить ключевую ставку 20 17,1,банк россии решил снизить ключевую ставку с 20...,банк россия решать снижать ключевой ставка,Совет директоров Банка России решил снизить кл...,совет директоров банка россии решил снизить кл...,совет директор банк россия решать снижать ключ...
3,мыс идокопас нато назвали первую цель начала п...,0,мыс идокопас в нато назвали первую цель для н...,мыс идокопас нато называть первый цель начало ...,В НАТО составили наступательный план действий ...,нато составили наступательный план действий за...,нато составлять наступательный план действие з...
4,украине прошли празднования 300 летия победы к...,0,на украине прошли празднования 300 летия побед...,украина проходить празднование летие победа ка...,Президент Владимир Зеленский принял в Полтаве ...,президент владимир зеленский принял полтаве па...,президент владимир зеленский принимать полтава...
5,минобороны рф заявило ударах ракетами «калибр»...,1,минобороны рф заявило об ударах ракетами «кали...,минобороны заявлять удар ракета калибр запорож...,"Минобороны России объявило, что ракетами больш...",минобороны россии объявило ракетами большой да...,минобороны россия объявлять ракета большой дал...
...,...,...,...,...,...,...,...
4404,чиновница призвавшая «найти покарать предателе...,0,чиновница призвавшая «найти и покарать предате...,чиновница призывать находить покарать предател...,Заместитель министра просвещения Ирина Карепин...,заместитель министра просвещения ирина карепин...,заместитель министр просвещение ирина карепин ...
4405,«включим программу киселёва покажем телевизора...,0,«включим программу киселёва и покажем телевизо...,включать программа киселев показывать телевизо...,Глава штабов Навального Леонид Волков призвал ...,глава штабов навального леонид волков призвал ...,глава штаб навальный леонид волк призывать сво...
4406,китайские власти могут казнить задержанного по...,0,китайские власти могут казнить задержанного по...,китайский власть мочь казнить задержанный подо...,Один из менеджеров стройки космодрома Восточны...,менеджеров стройки космодрома восточный взят с...,менеджер стройка космодром восточный взять стр...


In [ ]:
data_test['Stance'] = data_test['label'] # Переименуем целевую метрику
data_test.drop('label', axis=1, inplace=True)
data_test

,Body ID,Headline,Headline1,Headline2,articleBody,articleBody1,articleBody2,Stance
0,1,"['лукашенко', 'пригрозил', 'литовским', 'танка...",лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозить литовский танк белорусски...,"Президент Беларуси Александр Лукашенко заявил,...",президент беларуси александр лукашенко заявил ...,президент беларусь александр лукашенко заявлят...,0
1,2,"['российские', 'компании', 'оказались', 'опасн...",российские компании оказались в опасности из з...,российский компания оказываться опасность глоб...,Несколько крупнейших российских компаний оказа...,несколько крупнейших российских компаний оказа...,несколько крупный российский компания оказыват...,1
2,3,"['лукашенко', 'объявил', 'отмене', 'выборов', ...",лукашенко объявил об отмене выборов из за напа...,лукашенко объявлять отмена выборы нападение ин...,Президент Белоруссии Александр Лукашенко объяв...,президент белоруссии александр лукашенко объяв...,президент белоруссия александр лукашенко объяв...,0
3,4,"['«роснефть»', 'стала', 'лидером', 'объему', '...",«роснефть» стала лидером по объему биржевых пр...,роснефть становиться лидер объем биржевой прод...,С начала июля нефтяные компании заметно увелич...,начала июля нефтяные компании заметно увеличил...,начинать июль нефтяной компания заметно увелич...,1
4,5,"['псковской', 'области', 'дадут', 'десятки', '...",псковской области дадут десятки миллионов рубл...,псковский область давать десяток миллион рубль...,Псковская область получит 189 миллионов рублей...,псковская область получит 189 миллионов рублей...,псковский область получать миллион рубль ремон...,1
...,...,...,...,...,...,...,...,...
1096,1097,"['борис', 'джонсон', 'пообещал', 'избирателям'...",борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещать избиратель вернуть ирл...,Кандидат в премьер-министры Великобритании Бор...,кандидат премьер министры великобритании борис...,кандидат премьер министр великобритания борис ...,0
1097,1098,"['михаил', 'горбачев', 'заявил', 'подавал', 'о...",михаил горбачев заявил что не подавал в отстав...,михаил горбачев заявлять подавать отставка пос...,Сенсационное заявление сделал 8 декабря в эфир...,сенсационное заявление сделал 8 декабря эфире ...,сенсационный заявление сделать декабрь эфир не...,0
1098,1099,"['мгимо', 'появится', 'факультет', 'международ...",в мгимо появится факультет международных отнош...,мгимо появляться факультет международный отнош...,Московский государственный институт международ...,московский государственный институт международ...,московский государственный институт международ...,0
1099,1100,"['россии', 'введут', 'штраф', 'отрицание', 'аг...",в россии введут штраф за отрицание агрессии нато,россия вводить штраф отрицание агрессия нато,На рассмотрение в Государственную думу поступи...,рассмотрение государственную думу поступил зак...,рассмотрение государственный дума поступать за...,0


In [ ]:
# Сделаем столбец "Body ID" индексом
data_test = data_test.set_index("Body ID")
data_test

,Headline,Headline1,Headline2,articleBody,articleBody1,articleBody2,Stance
Body ID,,,,,,,
1,"['лукашенко', 'пригрозил', 'литовским', 'танка...",лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозить литовский танк белорусски...,"Президент Беларуси Александр Лукашенко заявил,...",президент беларуси александр лукашенко заявил ...,президент беларусь александр лукашенко заявлят...,0
2,"['российские', 'компании', 'оказались', 'опасн...",российские компании оказались в опасности из з...,российский компания оказываться опасность глоб...,Несколько крупнейших российских компаний оказа...,несколько крупнейших российских компаний оказа...,несколько крупный российский компания оказыват...,1
3,"['лукашенко', 'объявил', 'отмене', 'выборов', ...",лукашенко объявил об отмене выборов из за напа...,лукашенко объявлять отмена выборы нападение ин...,Президент Белоруссии Александр Лукашенко объяв...,президент белоруссии александр лукашенко объяв...,президент белоруссия александр лукашенко объяв...,0
4,"['«роснефть»', 'стала', 'лидером', 'объему', '...",«роснефть» стала лидером по объему биржевых пр...,роснефть становиться лидер объем биржевой прод...,С начала июля нефтяные компании заметно увелич...,начала июля нефтяные компании заметно увеличил...,начинать июль нефтяной компания заметно увелич...,1
5,"['псковской', 'области', 'дадут', 'десятки', '...",псковской области дадут десятки миллионов рубл...,псковский область давать десяток миллион рубль...,Псковская область получит 189 миллионов рублей...,псковская область получит 189 миллионов рублей...,псковский область получать миллион рубль ремон...,1
...,...,...,...,...,...,...,...
1097,"['борис', 'джонсон', 'пообещал', 'избирателям'...",борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещать избиратель вернуть ирл...,Кандидат в премьер-министры Великобритании Бор...,кандидат премьер министры великобритании борис...,кандидат премьер министр великобритания борис ...,0
1098,"['михаил', 'горбачев', 'заявил', 'подавал', 'о...",михаил горбачев заявил что не подавал в отстав...,михаил горбачев заявлять подавать отставка пос...,Сенсационное заявление сделал 8 декабря в эфир...,сенсационное заявление сделал 8 декабря эфире ...,сенсационный заявление сделать декабрь эфир не...,0
1099,"['мгимо', 'появится', 'факультет', 'международ...",в мгимо появится факультет международных отнош...,мгимо появляться факультет международный отнош...,Московский государственный институт международ...,московский государственный институт международ...,московский государственный институт международ...,0


In [ ]:
# Распарсим столбец "Headline"
data_test['Headline'] = data_test['Headline'].apply(lambda x: "".join(x.replace("[", "").replace("]", "").replace("'", "").split(',')))
data_test

,Headline,Headline1,Headline2,articleBody,articleBody1,articleBody2,Stance
Body ID,,,,,,,
1,лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозить литовский танк белорусски...,"Президент Беларуси Александр Лукашенко заявил,...",президент беларуси александр лукашенко заявил ...,президент беларусь александр лукашенко заявлят...,0
2,российские компании оказались опасности глобал...,российские компании оказались в опасности из з...,российский компания оказываться опасность глоб...,Несколько крупнейших российских компаний оказа...,несколько крупнейших российских компаний оказа...,несколько крупный российский компания оказыват...,1
3,лукашенко объявил отмене выборов нападения ино...,лукашенко объявил об отмене выборов из за напа...,лукашенко объявлять отмена выборы нападение ин...,Президент Белоруссии Александр Лукашенко объяв...,президент белоруссии александр лукашенко объяв...,президент белоруссия александр лукашенко объяв...,0
4,«роснефть» стала лидером объему биржевых прода...,«роснефть» стала лидером по объему биржевых пр...,роснефть становиться лидер объем биржевой прод...,С начала июля нефтяные компании заметно увелич...,начала июля нефтяные компании заметно увеличил...,начинать июль нефтяной компания заметно увелич...,1
5,псковской области дадут десятки миллионов рубл...,псковской области дадут десятки миллионов рубл...,псковский область давать десяток миллион рубль...,Псковская область получит 189 миллионов рублей...,псковская область получит 189 миллионов рублей...,псковский область получать миллион рубль ремон...,1
...,...,...,...,...,...,...,...
1097,борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещать избиратель вернуть ирл...,Кандидат в премьер-министры Великобритании Бор...,кандидат премьер министры великобритании борис...,кандидат премьер министр великобритания борис ...,0
1098,михаил горбачев заявил подавал отставку поста ...,михаил горбачев заявил что не подавал в отстав...,михаил горбачев заявлять подавать отставка пос...,Сенсационное заявление сделал 8 декабря в эфир...,сенсационное заявление сделал 8 декабря эфире ...,сенсационный заявление сделать декабрь эфир не...,0
1099,мгимо появится факультет международных отношен...,в мгимо появится факультет международных отнош...,мгимо появляться факультет международный отнош...,Московский государственный институт международ...,московский государственный институт международ...,московский государственный институт международ...,0


In [ ]:
# Подкачка словаря слов
nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Данные готовы для передачи в модели

# 2.1 Обучение моделей "из коробки"

In [ ]:
def fit_predict_score_model(df, model, df_test):
    """
    Input:
        df - тренировочный датафрейм с заголовком и текстом новости
        model - модель машинного обучения
        df_test - тестовый датафрейм с заголовком и текстом новости
    Return:
        Первым параметром возвращаем точность классификации модели на заголовках новостей
        Вторым параметром возвращаем точность классификации модели на содержании новостей
    """
    vectorizer = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=russian_stopwords)
    columns = df.drop('Stance', axis=1).columns # Получаем список признаков
    # print("-"*30)
    # print(f"   Пара признаков: {columns[0]} и {columns[1]}")

    # Обучаем TfidfVectorizer на всех новостях
    vectorizer.fit(df[columns[0]].values.astype('U'))
    vectorizer.fit(df[columns[1]].values.astype('U'))
    vectorizer.fit(df_test[columns[0]].values.astype('U'))
    vectorizer.fit(df_test[columns[1]].values.astype('U'))

    # Трансформируем текст в массив id-шников слов
    X11 = vectorizer.transform(df[columns[0]].values.astype('U')).toarray()
    X12 = vectorizer.transform(df[columns[1]].values.astype('U')).toarray()
    X21 = vectorizer.transform(df_test[columns[0]].values.astype('U')).toarray()
    X22 = vectorizer.transform(df_test[columns[1]].values.astype('U')).toarray()

    # Обучаем модель на заголовках новостей и смотрим на точность
    # print('   Обучение на заголовках')
    model_new = model
    model_new.fit(X11, df.Stance)
    preds1 = model_new.predict(X21)
    # print(f"      Точность на заголовках: {accuracy_score(df_test.Stance, preds1)}")

    # Обучаем модель на содержание новостей и смотрим на точность
    # print('   Обучение на содержание')
    model_new = model
    model_new.fit(X12, df.Stance)
    preds2 = model_new.predict(X22)
    # print(f"      Точность на содержаниях: {accuracy_score(df_test.Stance, preds2)}")

    return accuracy_score(df_test.Stance, preds1), accuracy_score(df_test.Stance, preds2)

In [ ]:
# Формируем DataFrame для записей точности моделей
accuracy_df = pd.DataFrame(columns=['Name Model', 'Headline', 'articleBody', 'Headline1', 'articleBody1', 'Headline2', 'articleBody2'])

i = 0
for model in [LogisticRegression(), RandomForestClassifier(), DecisionTreeClassifier(), SVC()]:
    # Словарь для заполнения в DataFrame точностей по модели
    accuracy_list = {'Name Model':None, 'Headline':None, 'articleBody':None, 'Headline1':None, 'articleBody1':None,
                     'Headline2':None, 'articleBody2':None}
    # print("*"*30)
    # print(model.__class__.__name__)
    # print("*"*30)

    # Тренируем модель на оригинальных данных и записываем в словарь
    itog1, itog2 = fit_predict_score_model(data[['Headline', 'articleBody', 'Stance']], model, data_test[['Headline', 'articleBody', 'Stance']])
    accuracy_list['Name Model'] = model.__class__.__name__
    accuracy_list['Headline'] = itog1
    accuracy_list['articleBody'] = itog2

    # Тренируем модель на данных после чистки и записываем в словарь
    itog1, itog2 = fit_predict_score_model(data[['Headline1', 'articleBody1', 'Stance']], model, data_test[['Headline1', 'articleBody1', 'Stance']])
    accuracy_list['Headline1'] = itog1
    accuracy_list['articleBody1'] = itog2

    # Тренируем модель на данных после обработки модулем PyMystem3 и записываем в словарь
    itog1, itog2 = fit_predict_score_model(data[['Headline2', 'articleBody2', 'Stance']], model, data_test[['Headline2', 'articleBody2', 'Stance']])
    accuracy_list['Headline2'] = itog1
    accuracy_list['articleBody2'] = itog2

    # Записываем данные по модели в датафрейм
    accuracy_df = pd.concat([accuracy_df, pd.DataFrame(accuracy_list, index=[i])], ignore_index=True)
    i += 1

Информация по точности всех моделей "из коробки" для каждых типов данных новостей получены

# 2.2 Выявление модели с лучшей точностью

In [ ]:
accuracy_df = accuracy_df.set_index('Name Model') # Меняем индексы на название моделей

In [ ]:
accuracy_df # Смотрим на данные

,Headline,articleBody,Headline1,articleBody1,Headline2,articleBody2
Name Model,,,,,,
LogisticRegression,0.751135,0.936421,0.751135,0.936421,0.785649,0.931880
RandomForestClassifier,0.739328,0.929155,0.743869,0.924614,0.797457,0.924614
DecisionTreeClassifier,0.689373,0.821980,0.698456,0.821980,0.727520,0.822888
SVC,0.752044,0.936421,0.751135,0.936421,0.801090,0.941871


In [ ]:
# Сравниваем среднюю точность модели по каждому из признаков
round(accuracy_df.mean().sort_values(ascending=0), 3)

articleBody     0.906
articleBody2    0.905
articleBody1    0.905
Headline2       0.778
Headline1       0.736
Headline        0.733
dtype: float64

Можно увидеть, что оригинальное содержание новостей после чистки даёт больше всего информативности, что помогает более точно классифицировать достоверность

In [ ]:
# Сравниваем точность модели по более информативному признаку
round(accuracy_df['articleBody1'].sort_values(ascending=0)*100, 3)

Name Model
LogisticRegression        93.642
SVC                       93.642
RandomForestClassifier    92.461
DecisionTreeClassifier    82.198
Name: articleBody1, dtype: float64

Лучший показатель точности "из коробки" показала модель SVC. Именно её будет лучше обучать

# 2.3 Обучение гиперпараметров у лучшей модели

In [ ]:
# Гиперпараметры для обучения
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

# Инициализируем модель с автоподбором гиперпараметров
clf = GridSearchCV(estimator=SVC(), param_grid=param_grid, cv=3,  n_jobs=-1)

In [ ]:
# Предобрабатываем данные для модели
vectorizer = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=russian_stopwords)
vectorizer.fit(data['articleBody1'].values.astype('U'))
vectorizer.fit(data_test['articleBody1'].values.astype('U'))
X_train = vectorizer.transform(data['articleBody1'].values.astype('U')).toarray()
X_test = vectorizer.transform(data_test['articleBody1'].values.astype('U')).toarray()

In [ ]:
%%time
clf.fit(X_train, data.Stance) # Обучаем все возможные модели
svc_best = clf.best_estimator_ # Получаем лучший экземпляр
svc_best.fit(X_train, data.Stance) # Обучаем лучший экземляр
preds = svc_best.predict(X_test) # Предсказываем
print(f"Точность на тесте: {round(accuracy_score(data_test.Stance, preds), 3)}")

Точность на тесте: 0.941
CPU times: user 19 s, sys: 696 ms, total: 19.7 s
Wall time: 5min 35s


Как итог, мы смогли добиться точности в 94 %